loading of assets and analysis of dataset

In [1]:
from io import StringIO, BytesIO
import folium
import folium.plugins
import pandas as pd
bus_icon = folium.CustomIcon(
    "assets/bus_stop.png",
    icon_size=(38, 95),
    icon_anchor=(22, 94),
    shadow_size=(50, 64),
    shadow_anchor=(4, 62),
    popup_anchor=(-3, -76),
)

cloud = folium.Icon(icon="cloud")

stops = pd.read_csv('GTFS-Daten/stops.txt')
print(stops.columns)


Index(['stop_id', 'stop_code', 'stop_name', 'stop_desc', 'stop_lat',
       'stop_lon', 'zone_id', 'stop_url', 'location_type', 'parent_station',
       'platform_code', 'stop_timezone_header'],
      dtype='object')


Creation of folium map and adding of feature group 'bus'

In [ ]:
m = folium.Map(location=(48.7624060,11.4257570), zoom_start=16)
stops_modified = stops[['stop_id','stop_lat','stop_lon','stop_name']]
# folium.Marker(location=[48.7624060,11.4257570], icon=cloud).add_to(m) 


fg = folium.FeatureGroup(name="groups")
m.add_child(fg)


for i,row in stops_modified.iterrows():
    folium.Marker(location=[row['stop_lat'],row['stop_lon']]).add_to(fg) 
    print([row['stop_lat'],row['stop_lon']])

folium.LayerControl(collapsed=False).add_to(m)

# m

cleaning of dataset to exclude parent stations and only the child stations

In [10]:
#create a dictionary {stop_name: [stop_lat,stop_lon]}
#create JSON

import json

stop_dict = {}

stops = pd.read_csv('GTFS-Daten/stops.txt')
parent_station_list=stops['parent_station'].unique()
filtered_df = stops[~stops['stop_id'].isin(parent_station_list)][['stop_id','stop_lat','stop_lon', 'stop_name']]
print(filtered_df)
# stops_modified = stops[['stop_id','stop_lat','stop_lon', 'stop_name']].loc[stops_modified['stop_id'].str[0] != "V"]


# json_stops = json.dumps(stop_dict, indent=4)

# with open("jstops.json", "w") as outfile:
#     outfile.write(json_stops)

# print(stops_modified)

      stop_id   stop_lat   stop_lon                    stop_name
1         101  48.739588  11.436405          Klinik - Dr. Reiser
2         102  48.739497  11.436552          Klinik - Dr. Reiser
4         201  48.776183  11.330453       Dünzlau, An der Breite
5         202  48.776540  11.329962       Dünzlau, An der Breite
7         301  48.718707  11.371754           Rosenschwaigstraße
...       ...        ...        ...                          ...
3400  1008401  48.412610  11.469910  Petershausen, S-Bahnhof Ost
3402  1008501  49.108716  11.350092          Greding, Viehhausen
3403  2012001  48.869148  11.065703      Dollnstein, Schlehenweg
3404  2013201  48.836810  11.021410                      Ensfeld
3405  2013301  48.856082  11.040198                    Haunsfeld

[2367 rows x 4 columns]


Testing of difflab library to check for closest string match for specific use cases:

e.g: augsburger straße geisenfeldwinden is on the xml but not a valid bus stop in trips.txt in GTFS. This is because its called augsburger straße geisenfeld instead.


This has to be implemented when a stop cant be found on stops.txt when merging with the xml

In [4]:
import difflib
print(difflib.get_close_matches('Geisenfeld, Augsburger Straße',['Geisenfeldwinden, Augsburger Str','Schrobenhausen, Augsburger Straße'],n=1))

['Geisenfeldwinden, Augsburger Str']


m2 is a test about marker clusters

In [ ]:
m2 = folium.Map(location=(48.7624060,11.4257570), zoom_start=16)

fg = folium.FeatureGroup(name="bus stop view")
fg2 = folium.FeatureGroup(name="live bus view")

m2.add_child(fg)
m2.add_child(fg2)

folium.LayerControl(collapsed=False, control=True).add_to(m2)


stops_cluster = folium.plugins.MarkerCluster(
    name='bus_stops',
    overlay=True,
    control=False,
    icon_create_function=None,
    
).add_to(fg)



for i,row in stops_modified.iterrows():
    marker = folium.Marker(location=[row['stop_lat'],row['stop_lon']], name=row['stop_name'], tooltip=row['stop_name'])
    stops_cluster.add_child(marker)


stops_search = folium.plugins.Search(
    layer=stops_cluster,
    geom_type="Point",
    placeholder="Search for a Bus Stop",
    collapsed=True,
    search_label="name",
).add_to(m2)

# m2
# m2.save("map.html")

getting current bus location data

In [ ]:
import xmltodict

xml_file = "SIRI_VM_Test_VGI.xml"

with open(xml_file, "r", encoding="ISO-8859-1") as f:
    xml_data = f.read()

# Convert XML to a Python dictionary
dict_data = xmltodict.parse(xml_data)["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"]["VehicleActivity"]

live_clusters = folium.plugins.MarkerCluster(
    name='bus_stops',
    overlay=True,
    control=False,
    icon_create_function=None,
).add_to(fg2)

for i in dict_data:
    marker = folium.Marker(location=[i['MonitoredVehicleJourney']['VehicleLocation']['Longtitude'],i['MonitoredVehicleJourney']['VehicleLocation']['Latitude']], name=)
    stops_cluster.add_child(marker)



stops_search = folium.plugins.Search(
    layer=live_clusters,
    geom_type="Point",
    placeholder="Search for a Bus",
    collapsed=True,
    search_label="name",
).add_to(m2)


{'@srsName': 'real', 'Longitude': '11.503959', 'Latitude': '48.711247'}


In [ ]:
n=12
print(dict_data[n]['MonitoredVehicleJourney']['PublishedLineName'])
print(dict_data[12]['MonitoredVehicleJourney']['DestinationName'])
print(dict_data[0]['MonitoredVehicleJourney']['Delay'])
print(dict_data[12]['Extensions']['init-o:OccupancyData']['init-o:PassengersNumber'])
print(dict_data[4]['Extensions']['init-o:OccupancyData']['init-o:OccupancyPercentage'])

INVG16
Zell, Dekan-Trost-Straße
PT4S
0
0
